In [1]:
import numpy as np
import pickle

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import RMSprop, Adam
from keras.utils import np_utils

Using TensorFlow backend.
/Users/bfeng/anaconda3/envs/ml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/bfeng/anaconda3/envs/ml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/bfeng/anaconda3/envs/ml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/bfeng/anaconda3/envs/ml/l

## Data

In [2]:
fn = 'data/shakespeare.txt'
sentences = []
all_chars = []
with open(fn, 'r') as f:
    for line in f:
        if len(line.strip()) > 3:
            if line[:2] == '  ': line = line[2:]
            sentences.append(line.lower())
            all_chars += list(line.lower())

In [3]:
# Create vocabulary.
chars = sorted(list(set(all_chars)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

n_chars = len(all_chars)
n_vocab = len(chars)
print('Total characters: %d' % n_chars)
print('Total vocab: %d' % n_vocab)

Total characters: 93673
Total vocab: 38


In [4]:
# Construct dataset.
seq_len = 40
step_size = 3
dataX = []
dataY = []
for i in range(0, n_chars - seq_len, step_size):
    seq_in = all_chars[i: i + seq_len]
    seq_out = all_chars[i + seq_len]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print('Total patterns: %d' % n_patterns)

Total patterns: 31211


In [5]:
X = np.reshape(dataX, (n_patterns, seq_len, 1))
X = X / float(n_vocab)
y = np_utils.to_categorical(dataY)

## Train model

In [6]:
dim = 200
model = Sequential()
model.add(LSTM(dim, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(y.shape[1], activation='softmax'))
optimizer = Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['acc'])
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 200)               161600    
_________________________________________________________________
dense_1 (Dense)              (None, 38)                7638      
Total params: 169,238
Trainable params: 169,238
Non-trainable params: 0
_________________________________________________________________
None


In [7]:
fn = 'lstm_weights/best_model
.h5' % dim
mc = ModelCheckpoint(
    'lstm_weights/best_model_dim-%d.h5' % dim, monitor='acc', save_best_only=True)
es = EarlyStopping(monitor='acc', baseline=0.6, patience=0)
callbacks_list = [mc]

In [13]:
model.fit(X, y, epochs=200, batch_size=128, callbacks=callbacks_list, initial_epoch=75)

Epoch 76/200
31211/31211 [==============================] - 13s 423us/step - loss: 1.2783 - acc: 0.6019
Epoch 77/200
31211/31211 [==============================] - 14s 441us/step - loss: 1.2739 - acc: 0.6006
Epoch 78/200
31211/31211 [==============================] - 14s 436us/step - loss: 1.3241 - acc: 0.5865
Epoch 79/200
31211/31211 [==============================] - 14s 446us/step - loss: 1.2586 - acc: 0.6054
Epoch 80/200
31211/31211 [==============================] - 13s 432us/step - loss: 1.2314 - acc: 0.6144
Epoch 81/200
31211/31211 [==============================] - 13s 429us/step - loss: 1.2212 - acc: 0.6197
Epoch 82/200
31211/31211 [==============================] - 14s 434us/step - loss: 1.2377 - acc: 0.6140
Epoch 83/200
31211/31211 [==============================] - 13s 431us/step - loss: 1.2305 - acc: 0.6130
Epoch 84/200
31211/31211 [==============================] - 13s 430us/step - loss: 1.2149 - acc: 0.6181
Epoch 85/200
31211/31211 [==============================] - 13s 

31211/31211 [==============================] - 14s 434us/step - loss: 0.9015 - acc: 0.7138
Epoch 155/200
31211/31211 [==============================] - 14s 456us/step - loss: 0.8842 - acc: 0.7172
Epoch 156/200
31211/31211 [==============================] - 15s 474us/step - loss: 0.8941 - acc: 0.7171
Epoch 157/200
31211/31211 [==============================] - 14s 437us/step - loss: 0.9409 - acc: 0.6999
Epoch 158/200
31211/31211 [==============================] - 14s 439us/step - loss: 0.8795 - acc: 0.7213
Epoch 159/200
31211/31211 [==============================] - 13s 430us/step - loss: 0.8648 - acc: 0.7219
Epoch 160/200
31211/31211 [==============================] - 13s 430us/step - loss: 0.8231 - acc: 0.7379
Epoch 161/200
31211/31211 [==============================] - 13s 425us/step - loss: 0.8580 - acc: 0.7263
Epoch 162/200
31211/31211 [==============================] - 13s 429us/step - loss: 0.8726 - acc: 0.7228
Epoch 163/200
31211/31211 [==============================] - 13s 428u

## Generate text

In [ ]:
def sample(preds, temperature=1.0):
    # Helper function to sample from a probability array
    # Source: https://keras.io/examples/lstm_text_generation/
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [14]:
ckpt = "lstm_weights/best_model_dim-%d.h5" % dim
model.load_weights(ckpt)
model.evaluate(X, y)

31211/31211 [==============================] - 7s 226us/step


[0.6564503235568844, 0.7918041944503784]

In [15]:
seed = "shall i compare thee to a summer's day?\n"
pattern = [char_to_int[char] for char in seed]
output_seq = pattern

In [16]:
tmp = 1.0
for i in range(1000):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    #index = sample(prediction, temperature=tmp)
    index = np.argmax(prediction)
    pattern.append(index)
    pattern = pattern[1:]
    output_seq.append(index)

In [17]:
output = [int_to_char[i] for i in output_seq]
output = ''.join(output)
print(output)

shall i compare thee to a summer's day?
tthos aaiut mr haart in thls thruldst,
io ttcv gile tonl cidutee't cum for the sreeser ofrhohgs bdyint sh thene,
wiich bllnger she shaho, when sookts uhou art alvh mfnd own lose,
wiech nno mieyhnl au dnrld io theme bgnrned
oo thee,
to mons of lensr in the grared bod shee.
when sookir't sooeu caste ost live iy bs g lov medru
cyp his bdcrhe,
and hos lemea bla esrh laspen bs darphlc oossoen bsleg,
thene bllpe ho the faatini oagenad,
then thou iolwinla bnd wht aets mn theer gour facr stame.
bnd wou and oanedr thmu tha wirrh soeil,
fne poicren, bnd shmu art io thy beutrees bat iise,
and sha derter as iy soaiterg sioguw
shael non.
thel thet makd that thec foieers our pfgwmyh,
that shmuld yhth mooe aeloneit amun,
orrt shat ph your frart loot dnthwued shre,
iow maty ay thes mare bla oae cnoteerted brd shee.
when sookei's kunw thou mott laee then i poeide yith that is ulge
iilvllg ladk,
the bglogn that ceiuiy ceauty shoulds delrh.
and for the gaglpred ios

In [12]:
model.evaluate(X, y)

31211/31211 [==============================] - 7s 237us/step


[1.2026776160613595, 0.6283681988716125]